In [ ]:
import re
import urllib
import json
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import torch.nn as nn
import texthero as hero
from urllib.parse import urlsplit
import requests
import torch
from bs4 import BeautifulSoup
from requests import get
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from huggingface_hub import from_pretrained_keras
import spacy_sentence_bert
import csv

In [ ]:
from summarizer import Summarizer,TransformerSummarizer

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from timeit import default_timer as timer

### TO DO
1. News section search - Done
2. Abstarctive summary
3. Specify cap on number of links
4. Search Keywords
5. Tune similarity parameter n Model
6. Use summary or not

## Search Google News

In [ ]:
import math
from GoogleNews import GoogleNews
from datetime import datetime

In [ ]:
googlenews = GoogleNews(start='12/31/2019',end='01/01/2023', lang='en')
googlenews.set_encode('utf-8')

In [ ]:
def get_links_from_google_news(claim, top=10):
    googlenews.search(claim)
    
    results = googlenews.results()
    googlenews.clear()
    
#     results = [x for x in results if isinstance(x['datetime'], datetime)]
    
#     def func(ele):
#         return str(ele['datetime'])
    
#     results.sort(key = func, reverse = True)
    links = [x['link'] for x in results]
    
    return links

## Scraping Evidences

In [ ]:
nlp = spacy_sentence_bert.load_model('en_nli_roberta_base')

In [ ]:
def get_sentences_from_link(link, text, top=10):
    request = requests.get(link, verify=False, timeout=20)
    Soup = BeautifulSoup(request.text, 'lxml')
    
    heading_tags = ['p']


    results = []
    used = []

    for tags in Soup.find_all(heading_tags):
        if 'h' in tags.name:
            tokens = tags.text.strip().split()
            if len(tokens) > 8:
                if tags.text.strip() not in used:
                    used.append(tags.text.strip())
                    results.append([tags.name, tags.text.strip()])
        else:
            tokens = tags.text.strip().split()
            if len(tokens) > 8:
                if tags.text.strip() not in used:
                    used.append(tags.text.strip())
                    results.append([tags.name, tags.text.strip()])
    doc1 = nlp(text)
    sim = []
    for r in results:
        sim.append(doc1.similarity(nlp(r[1])))
    zipped = zip(sim, results)
    zipped = sorted(zipped, reverse=True)
    high_conf = [a for s, a in zipped if s >= 0.6]

    return high_conf[:top], request.url

In [ ]:
def scrap_evidences(text, links):
    new_links = []
    for link in links:
        conf, lin = get_sentences_from_link(link, text)
        new_links.append([lin, conf])
    return new_links

## Concatinating Evidences

In [ ]:
def concatenate_evidences(claim, links):
    summ = []
    for link in links:
        if type(link[1]) == list:
            for text in link[1]:
                if type(link[1]) == list:
                    summ.append(text[1])
                else:
                    summ.append(text)
        elif type(link[1]) == str:
            summ.append(link[1])

    urls = re.findall(r'https?:\/+\/+t+\.+co+\/+\S*', claim)
    
    for li in urls:
        claim = claim.replace(li, '')
    claim = claim.strip()

    if summ:
        summary = (claim, ' '.join(summ).replace('\n', '').replace('\t', ''))
    else:
        summary = ('', '')

    return summary

## Extractive Summary

In [ ]:
# summarizer = Summarizer()

In [ ]:
# def generate_extractive_summary(evidence):
#     return ''.join(summarizer(evidence, min_length=60))

## Abstractive Summary

In [ ]:
from transformers import pipeline
import os

In [ ]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# abstractive_summarizer = pipeline("summarization")

In [ ]:
# abstractive_summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

In [ ]:
# def generate_abstractive_summary(evidence):
#     return abstractive_summarizer(evidence, min_length=5, do_sample=False)[0]['summary_text']

## Semantic Based Similarity

In [ ]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

In [ ]:
model = from_pretrained_keras("keras-io/bert-semantic-similarity")
labels = ["contradiction", "entailment", "neutral"]

In [ ]:
def predict(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = model.predict(test_data[0])[0]
    
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return labels_probs

## Fake News Detection

In [ ]:
def fake_news_detection(claim):
#     start = timer()
    links = get_links_from_google_news(claim)
#     print("links ----- ")
#     print(links)
    evidence_list = scrap_evidences(claim, links[:10])
#     print("Evidence List ----- ")
#     print(evidence_list)
    evidence = concatenate_evidences(claim, evidence_list)
#     print("Concatenated Evidence ----- ")
#     print(evidence[1])
#     print("Results with Concatenated Evidence")
    k = predict(claim, evidence[1])
    print(k)
#     extractive_summary = generate_extractive_summary(evidence[1])
#     print("Extractive Summary ----- ")
#     print(extractive_summary)
#     print("Results with Extractive Summary")
#     print(predict(claim, extractive_summary))
#     abstractive_summary = generate_abstractive_summary(evidence[1])
#     print("Abstractive Summary ----- ")
#     print(abstractive_summary)
#     print("Results with Abstractive Summary")
#     print(predict(claim, abstractive_summary))
#     end = timer()
#     print(end - start)
#     return evidence[1]
    return (k,  evidence[1])

    

In [ ]:
# claim = "Times Of India Shares Fake Cropped Bungee Jump Fail Video"
# # claim = "People from LGBTQ Community can marry in india"
# v = fake_news_detection(claim)

In [ ]:
# print(v)

In [ ]:
# generate_abstractive_summary(v)

In [ ]:
data = pd.read_excel('IFND_dataset.xlsx')

In [ ]:
# data = data.sample(frac = 1)
# data = data.reset_index()
# data = data.drop(['index'], axis=1)

In [ ]:
data.head()

In [ ]:
filename = "data.csv"

In [ ]:
## VERY SENSITIVE PIECE OF CODE
# with open(filename, 'w') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerow(['id', 'contradiction', 'entailment', 'neutral', 'evidence'])


In [ ]:
with open(filename, 'a') as csvfile: 
    csvwriter = csv.writer(csvfile)
    for i in range(763, 800):
        print(i)
        try:
            v = fake_news_detection(data.iloc[i]['Statement'])
            csvwriter.writerow([data.iloc[i]['id'], v[0]['contradiction'], v[0]['entailment'], v[0]['neutral'], v[1]])
        except:
            print("failed at " + str(i))

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df

,id,contradiction,entailment,neutral,evidence
0,1,0.000049,4.556509e-06,0.999947,* The amendment has added a provision to the o...
1,2,0.002970,7.918521e-05,0.996951,Prime Minister Narendra Modi will address the ...
2,4,0.324765,1.303815e-01,0.544854,NaN
3,6,0.898778,1.711081e-05,0.101205,Kerala Chief Minister Pinarayi Vijayan on Satu...
4,7,0.333958,3.113852e-01,0.354656,NaN
...,...,...,...,...,...
237,779,0.204934,6.528817e-01,0.142184,NaN
238,781,0.999408,4.236749e-07,0.000592,INEQUALITY: India ranks among the bottom 15 of...
239,782,0.002991,1.175789e-05,0.996997,"Continuing her offensive against Amit Shah, BJ..."
240,783,0.590170,2.313530e-01,0.178477,NaN


In [3]:
df = df.dropna()
df

,id,contradiction,entailment,neutral,evidence
0,1,0.000049,4.556509e-06,0.999947,* The amendment has added a provision to the o...
1,2,0.002970,7.918521e-05,0.996951,Prime Minister Narendra Modi will address the ...
3,6,0.898778,1.711081e-05,0.101205,Kerala Chief Minister Pinarayi Vijayan on Satu...
5,8,0.000078,1.241869e-06,0.999921,Goa government appealed to the people not to p...
6,9,0.034757,3.974534e-04,0.964845,"""The important question for all of us is - any..."
...,...,...,...,...,...
234,774,0.040354,5.125620e-05,0.959595,Mr Verma seems to be ignoring the fact that Ra...
235,777,0.069987,4.146308e-03,0.925867,“The minister has spoken about serving eggs to...
238,781,0.999408,4.236749e-07,0.000592,INEQUALITY: India ranks among the bottom 15 of...
239,782,0.002991,1.175789e-05,0.996997,"Continuing her offensive against Amit Shah, BJ..."


In [4]:
df = df.drop_duplicates(subset=['id'], keep='last')
df

,id,contradiction,entailment,neutral,evidence
0,1,0.000049,4.556509e-06,0.999947,* The amendment has added a provision to the o...
1,2,0.002970,7.918521e-05,0.996951,Prime Minister Narendra Modi will address the ...
3,6,0.898778,1.711081e-05,0.101205,Kerala Chief Minister Pinarayi Vijayan on Satu...
5,8,0.000078,1.241869e-06,0.999921,Goa government appealed to the people not to p...
6,9,0.034757,3.974534e-04,0.964845,"""The important question for all of us is - any..."
...,...,...,...,...,...
234,774,0.040354,5.125620e-05,0.959595,Mr Verma seems to be ignoring the fact that Ra...
235,777,0.069987,4.146308e-03,0.925867,“The minister has spoken about serving eggs to...
238,781,0.999408,4.236749e-07,0.000592,INEQUALITY: India ranks among the bottom 15 of...
239,782,0.002991,1.175789e-05,0.996997,"Continuing her offensive against Amit Shah, BJ..."


In [5]:
df.dtypes

id                 int64
contradiction    float64
entailment       float64
neutral          float64
evidence          object
dtype: object